In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from code.mpmath_series import quad_phi
from code.mpmath_series import series_x_mu_b0_pos, bound_series_x_mu_b0_pos, estimate_N_series_x_mu_b0_pos
from code.mpmath_series import series_x_mu_b0_alternating, bound_series_x_mu_b0_alternating, estimate_N_series_x_mu_b0_alternating

from code.mpmath_series import estimate_N_series_x_mu_b0_pos_fast, estimate_Tk_series_x_mu_b0_pos_fast

In [4]:
from mpmath import *

In [890]:
mp.dps = 50

alpha = mpf('5')
mu = mpf('1/5')
delta = mpf('1')
x = mpf('4')

(x - mu) / delta

omega = sqrt((x-mu)**2 + delta**2)
float(alpha * omega), float(((x-mu)**2  / omega * alpha)), float(omega**2)

(19.6468827043885, 18.374416207990283, 15.44)

### 1. Integral

In [891]:
ref = quad_phi(x, alpha, mu, delta, digits=100)
ref

mpf('0.99999999024325753057139053233962285723850702619936739')

### 2. Series

In [892]:
estimate_N_series_x_mu_b0_pos(x, alpha, mu, delta)

0.9670745372626464 5.813381734846124 9.418732088260908e-35 1.5202999468351408e+33 538.6482350660204


mpf('584.75015588819904015446256335785950141695240951570337')

In [893]:
bound_series_x_mu_b0_pos(x, alpha, mu, delta, 3458)

7.468998749658522e-101

In [92]:
float(ref - series_x_mu_b0_pos(x, alpha, mu, delta, 3458))

3.5488785725468425e-16

Series

In [652]:
omega = sqrt(delta**2 + (x-mu)**2)
z = (x-mu)**2 * alpha / omega

N = 18

s = mp.zero
for k in range(N):
    r = z**k / fac2(2*k + 1)
    q = besselk(k + 1, alpha * omega)
    s += r * q

remainder = mp.zero
for k in range(N, 151):
    r = z**k / fac2(2*k + 1)
    q = besselk(k + 1, alpha * omega)
    remainder += r * q

print(s, remainder)

0.48415015167516864127 2.6567777060793137811


Remainder integral representation

In [653]:
M = gamma(N + 3 / 2) * 2 / fac2(2*N + 1) * (2*alpha / omega) / sqrt(pi) * (2*z*alpha / omega)**N
M * quad(lambda t: cos(omega*t) / (t**2 + alpha**2)**(N + 3/2) / (2 - 2*z*alpha / omega / (t**2 + alpha**2)), [0, inf])

mpf('155.03203079677370037877')

In [809]:
mp.dps = 100

N = 73

omega = sqrt(delta**2 + (x-mu)**2)
z = (x-mu)**2 * alpha / omega
aw = alpha * omega

s = mp.zero
for k in range(N):
    r = z**k / fac2(2*k + 1)
    q = besselk(k + 1, alpha * omega)
    s += r * q

sN = s

In [791]:
m = 2*z*alpha/omega

S1 = gamma(N + 3 / 2) / fac2(2*N + 1) * (alpha / omega) * sqrt(pi) * 2**(N+1) / sqrt(2*alpha**2*m - m*m) * cosh(sqrt(omega**2 * (2*alpha**2 - m)) / sqrt(2))

S2 = 0
for k in range(N):
    P = alpha**(2*k) * gamma(N + 1 - k) * gamma(k - 1/2) * hyp2f1(1, N + 1 - k, 3/2 - k, 1 - m/(2*alpha**2))
    tk = (-1)**k * omega**(2*k) / factorial(2*k) * P
    S2 += tk
    print(k, tk)

S2 *= (m * 2)**N * alpha**(-2*N -1) / pi / fac2(2*N - 1) * gamma(N + 3 / 2) / fac2(2*N + 1) / omega

S = sN + S1 + S2

SF = S * delta * exp(delta * alpha) / pi  * (x-mu)*alpha / omega  + 1/2

0 -5.496243269871910122220601367354632863160099964038016328642899335268470411882641699112998443798002909e+110
1 -7.633681398409604803124266757282981322912688808062775301430975745091275856130949391371201533114168768e+112
2 -1.767055768526338854944453400682140504927173560229483896187064338287916642475628029881255610096002199e+114
3 -1.636162763278878667294492321534281893895758124602129938356900760440121220771588372644999431239728362e+115
4 -8.115886705077125093207108221980516488751572741976950293117414860716345931678326831660509636330185224e+115
5 -2.50490330589207334738071812473010218923448902218748386669546442316180669696345463194059410627938434e+116
6 -5.271261163850632268790530333645444112084890207154951365527597839817090881291592014944777254320056729e+116
7 -8.045270398077195601185983182145534830225725396183997759697869771267601827799400188192593254217107359e+116
8 -9.311655552189845500378633771170207022247878299735169001015197139544337913469733360936398923547393429e+116
9 -8.452846

In [792]:
SF

mpf('0.9999999999999356381660900636991506181051935971668067788365973953916764577449658345252909818536629515857')

In [793]:
ref

mpf('0.9999999999999356269605098964469005136041216274672227588182649266940140867312686304645352922804396326724')

In [794]:
float(ref -SF)

-1.120558016725225e-17

In [784]:
1 - m/(2*alpha**2)

mpf('0.1479289940828402366863905325443786982248520710059170334290690583500534893832768231329247435133297803884')

Find $N$

minimum value

In [894]:
mp.dps = 20

m = 2*z*alpha/omega

In [895]:
(alpha*omega)**(2*N) * gamma(N - 1/2) / factorial(2*N)

mpf('3600488529029.6351994313')

In [896]:
(alpha*omega)**(2*N) * 2**(-2*N - 1/2) * (e / N)**N

mpf('109069407681678.80307257')

In [897]:
(gamma(N - 1/2) * (alpha * omega)**(2*N) / factorial(2*N) *
 (m * 2)**N * alpha**(-2*N -1) / pi / fac2(2*N - 1) * gamma(N + 3 / 2) / fac2(2*N + 1) / omega)

mpf('277952548155.01822560234')

In [898]:
(gamma(N - 1/2) * (sqrt(2*m) * omega)**(2*N) / factorial(2*N)
 * alpha**(-1) / pi / fac2(2*N - 1) * gamma(N + 3 / 2) / fac2(2*N + 1) / omega)

mpf('277952548155.01822560234')

In [899]:
gamma(N - 1/2)  / fac2(2*N - 1) * gamma(N + 3 / 2) / fac2(2*N + 1) / factorial(2*N)

mpf('6.4814435302925893819162e-26')

In [900]:
sqrt(pi) * (e / N) **(2*N) * 2**(-4*N) / 4

mpf('1.9552610841160684826832e-24')

In [901]:
sqrt(pi) * (e / N) **(2*N) * 2**(-4*N - 2) * (m * 2)**N * alpha**(-2*N -1) / pi / omega * (alpha * omega)**(2*N) 

mpf('8385011735400.7250348255')

In [902]:
(sqrt(2 * m) * omega)**(2*N) / alpha / sqrt(pi) / 4 / omega * (e / N) **(2*N) * 2**(-4*N)

mpf('8385011735400.725034833')

In [903]:
bb = 4 * epsilon * (alpha * omega) * sqrt(pi)
xx = sqrt(2 * m) * omega

In [904]:
-log(bb) / 2 / lambertw(-2*log(bb) / (e * xx))

mpf('70.958796482812211387056')

In [612]:
epsilon = 1e-16
# (alpha * omega / 2)**(2*N) * (e / N)**N = epsilon * sqrt(2)

In [613]:
(alpha * omega / 2)**(2*N) * sqrt(2) * (e / N)**N

mpf('566841040.6641643685890213496249084067053109066700941309270893853776064560947925650594978161882493220779')

In [614]:
-log(epsilon * sqrt(2)) / lambertw(-log(epsilon * sqrt(2)) / e  / (alpha * omega / 2)**2)

mpf('84.87735328497126509775084285759390623338178067893380875677636449468878987773073805851855576020182298088')

In [528]:
1 - m/(2*alpha**2)

mpf('0.0000043402589398483517753292')